In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from shapely.geometry import Point

import contextily as ctx
#from geopy.distance import geodesic
from datetime import datetime


In [ ]:
filename_DD = '../data/nextbike/trips_Dresden 2025-01-26_19-59-31.pkl'
filename_FB = '../data/nextbike/trips_Freiburg 2025-01-26_19-59-31.pkl'
df_DD = pd.read_pickle(filename_DD)
df_FB = pd.read_pickle(filename_FB)

In [ ]:
original_cols = df_DD.columns

In [ ]:
len(df_DD)

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option('display.max_colwidth', 400)

In [ ]:
len(df_FB)

# locations too far

# DD

In [ ]:
df_DD.head()

In [ ]:
tmp = df_DD[[ "name_return", 'bike_return']].value_counts(dropna=False)

In [ ]:
tmp = df_DD[[ "name_rent", 'bike_rent']].value_counts(dropna=False)

In [ ]:
tmp.head(40)

In [ ]:
tmp.head(40)

In [ ]:
grouping = df_DD[(df_DD.uid_rent == df_DD.uid_return)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)

In [ ]:
print(grouping)

In [ ]:
outlier_days_DD = ["2024-02-12", "2024-02-13", "2024-02-14", "2024-03-27", "2024-09-19", "2024-09-20", "2024-09-22", "2024-09-24", "2024-10-09", "2024-10-27"]
outlier_days_DD = [pd.to_datetime(x).date() for x in outlier_days_DD]

In [ ]:
flt_DD = (df_DD.date_return.isin(outlier_days_DD)) & (df_DD.date_rent.isin(outlier_days_DD)) & (df_DD.uid_rent == df_DD.uid_return)  & (df_DD.duration_min<=60 )
df_DD[flt_DD].groupby("duration_min_bin").size().sort_values(ascending=False)

In [ ]:
len(df_DD[flt_DD])/len(df_DD)

In [ ]:
len(df_DD[flt_DD])

In [ ]:
df_DD = df_DD[~flt_DD]

In [ ]:
grouping = df_DD[(df_DD.uid_rent == df_DD.uid_return)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)


In [ ]:
flt_DD_2 = (df_DD.duration_min<=5) & (df_DD.uid_rent == df_DD.uid_return)
len(df_DD[flt_DD_2])/len(df_DD)

In [ ]:
len(df_DD[flt_DD_2])

In [ ]:
0.004175813250096153 + 0.011861378594605957

In [ ]:
df_DD = df_DD[~flt_DD_2]

In [ ]:
len(df_DD)

In [ ]:
# grouping

In [ ]:
# df_DD.groupby(['datetime_return', 'uid_return', 'name_return']).size().sort_values(ascending=False).head(40)

# FB

In [ ]:
df_FB.head()

In [ ]:
# already accounted for by 24 hour filter 
# missing_FB_adays = ["2023-06-26", "2023-06-27", "2023-06-29", "2023-06-30"]
# missing_FB_days = [pd.to_datetime(x).date() for x in missing_FB_days]

In [ ]:
# missing_FB_days

In [ ]:
#tmp = df_FB.groupby("date_return").size().sort_index()

In [ ]:
grouping = df_FB[(df_FB.uid_rent == df_FB.uid_return) & (df_FB.duration_min<=60)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)

In [ ]:
grouping[grouping > 10].head(30).sort_values(ascending=False)

In [ ]:
outlier_days_FB = ["2023-06-04", "2023-06-08",  "2023-06-24","2024-09-20"]
outlier_days_FB = [pd.to_datetime(x).date() for x in outlier_days_DD]

In [ ]:
flt_FB = (df_FB.date_return.isin(outlier_days_FB)) & (df_FB.date_rent.isin(outlier_days_FB)) & (df_FB.uid_rent == df_FB.uid_return)  & (df_FB.duration_min<=60 )
df_FB[flt_FB].groupby("duration_min_bin").size().sort_values(ascending=False)

In [ ]:
len(df_FB[flt_FB])/len(df_FB)*100

In [ ]:
len(df_FB[flt_FB])

In [ ]:
df_FB = df_FB[~flt_FB]

In [ ]:
len(df_FB)

In [ ]:
grouping = df_FB[(df_FB.uid_rent == df_FB.uid_return) & (df_FB.duration_min<=60)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)

In [ ]:
grouping[grouping > 10].head(30).sort_values(ascending=False)

In [ ]:
flt_FB_2 = (df_FB.duration_min<=5) & (df_FB.uid_rent == df_FB.uid_return)
len(df_FB[flt_FB_2])/len(df_FB)

In [ ]:
len(df_FB[flt_FB_2])

In [ ]:
0.034818869529613335 + 0.002905363626274317

In [ ]:
df_FB = df_FB[~flt_FB_2]

# location outliers

In [ ]:
df_FB[['lat_rent', 'lng_rent', 'lat_return', 'lng_return']].describe(percentiles=[0.001, 0.01, 0.05, 0.5, 0.95, 0.995])

In [ ]:
df_DD[['lat_rent', 'lng_rent', 'lat_return', 'lng_return']].describe(percentiles=[0.001, 0.01, 0.05, 0.5, 0.95, 0.995])

In [ ]:
station_coordinates= df_DD.loc[~df_DD.station_number_return.isna(), ['lat_return', 'lng_return']].drop_duplicates()

In [ ]:
station_coordinates

In [ ]:
len(station_coordinates)

In [ ]:
tree = KDTree(station_coordinates)

In [ ]:
return_points_to_filter = df_DD[['lat_return', 'lng_return']]
rent_points_to_filter = df_DD[['lat_rent', 'lng_rent']]

In [ ]:
distances_return, _ = tree.query(return_points_to_filter)
distances_rent, _ = tree.query(rent_points_to_filter)

In [ ]:
distances_return

In [ ]:
km_per_degree = 111.32

In [ ]:
distances_km_return = distances_return * km_per_degree
distances_km_rent = distances_rent * km_per_degree

In [ ]:
distances_km_return = pd.Series(distances_km_return)
distances_km_rent = pd.Series(distances_km_rent)

In [ ]:
len(distances_km_rent[distances_km_rent>1])

In [ ]:
len(distances_km_return[distances_km_return>1])

In [ ]:
flt_DD_3 = (distances_km_return > 1) | (distances_km_rent > 1)

In [ ]:
flt_DD_3.index

In [ ]:
df_DD = df_DD.reset_index(drop=True)

In [ ]:
len(df_DD[flt_DD_3])

In [ ]:
len(df_DD[~flt_DD_3])

In [ ]:
df_DD = df_DD[~flt_DD_3]

In [ ]:
len(df_DD)

### FB

In [ ]:
station_coordinates= df_FB.loc[~df_FB.station_number_return.isna(), ['lat_return', 'lng_return']].drop_duplicates()

In [ ]:
station_coordinates

In [ ]:
len(station_coordinates)

In [ ]:
tree = KDTree(station_coordinates)

In [ ]:
return_points_to_filter = df_FB[['lat_return', 'lng_return']]
rent_points_to_filter = df_FB[['lat_rent', 'lng_rent']]

In [ ]:
distances_return, _ = tree.query(return_points_to_filter)
distances_rent, _ = tree.query(rent_points_to_filter)

In [ ]:
distances_return

In [ ]:
km_per_degree = 111.32

In [ ]:
distances_km_return = distances_return * km_per_degree
distances_km_rent = distances_rent * km_per_degree

In [ ]:
distances_km_return = pd.Series(distances_km_return)
distances_km_rent = pd.Series(distances_km_rent)

In [ ]:
len(distances_km_rent[distances_km_rent>1])

In [ ]:
len(distances_km_return[distances_km_return>1])

In [ ]:
flt_FB_3 = (distances_km_return > 1) | (distances_km_rent > 1)

In [ ]:
flt_FB_3.index

In [ ]:
df_FB = df_FB.reset_index(drop=True)

In [ ]:
len(df_FB[flt_FB_3])

In [ ]:
len(df_FB[~flt_FB_3])

In [ ]:
df_FB = df_FB[~flt_FB_3]

In [ ]:
len(df_FB)

# Too fast and mass returns

In [ ]:
df_DD.head()
df_DD['speed_kmh'] = df_DD.distance/1000/df_DD.duration_min*60
#df_DD.speed_kmh.plot(kind='hist', bins=100, xlim=(0, 1000))
df_DD[df_DD.speed_kmh>50].groupby("name_return", dropna=False).size().sort_values(ascending=False)
len(df_DD[df_DD.speed_kmh>50])/len(df_DD)
#df_DD[df_DD.speed_kmh>50]

In [ ]:
#len(df_DD[df_DD.speed_kmh>50])

In [ ]:
#df_DD = df_DD[df_DD.speed_kmh<=50]

In [ ]:
df_DD[df_DD.station_number_return==43115]

In [ ]:
#grouping = df_DD[df_DD.name_return =="MOBIpunkt TU Dresden"].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
# df_DD[df_DD.datetime_return == pd.to_datetime("2024-03-27 15:02:00")] # tracking disturbance, but these are real trips, there was tracking with a delay
df_DD.groupby("name_return").size().sort_values(ascending=False).head(10)
df_DD.name_return.unique()

In [ ]:
grouping = df_DD[~df_DD.name_return.astype(str).str.contains("BIKE", na=False)].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
grouping[grouping>10]

In [ ]:
df_hbf = df_DD[df_DD.name_return.astype(str).str.contains("Haupt")].copy()

In [ ]:
df_hbf_counts = df_hbf.datetime_hour_return.value_counts().sort_index()

In [ ]:
df_hbf_counts = df_hbf_counts[df_hbf_counts>10]

In [ ]:
#grouping = grouping.reset_index()

In [ ]:
#grouping.datetime_return.dt.weekday.value_counts()

In [ ]:
df_DD

In [ ]:
grouping = grouping[grouping>10]

In [ ]:
# filter out the bike stations of df_DD based on the multi index
flt_DD = df_DD.set_index(["datetime_return", "name_return"]).index.isin(grouping.index)

df_DD['is_mass_return']=False
df_DD['is_too_fast']=False

df_DD.loc[flt_DD, "is_mass_return"]=True
df_DD.loc[df_DD.speed_kmh>50, "is_too_fast"]=True

df_DD[['is_too_fast', 'is_mass_return']].value_counts()
df_DD.groupby("is_mass_return")['speed_kmh'].mean()
# correlation of 2 boolean columns
df_DD[['is_too_fast', 'is_mass_return']].corr()
df_DD[df_DD.is_mass_return==True].groupby("name_return").size().sort_values(ascending=False)
df_DD['return_hour_int'] = df_DD.datetime_return.dt.hour
df_DD[df_DD.is_mass_return==True].groupby("return_hour_int").size().sort_index()
df_DD.is_mass_return.value_counts(normalize=True)

In [ ]:
df_FB.head()
df_FB['speed_kmh'] = df_FB.distance/1000/df_FB.duration_min*60
df_FB.speed_kmh.plot(kind='hist', bins=100, xlim=(0, 1000))
df_FB[df_FB.speed_kmh>50].groupby("name_return", dropna=False).size().sort_values(ascending=False)
len(df_FB[df_FB.speed_kmh>50])/len(df_FB)
df_FB[df_FB.speed_kmh>50]
grouping = df_FB[df_FB.name_return =="MOBIpunkt TU Dresden"].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
# df_FB[df_FB.datetime_return == pd.to_datetime("2024-03-27 15:02:00")] # tracking disturbance, but these are real trips, there was tracking with a delay
df_FB.groupby("name_return").size().sort_values(ascending=False).head(10)
df_FB.name_return.unique()
grouping = df_FB[~df_FB.name_return.astype(str).str.contains("BIKE")].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
grouping
grouping = grouping[grouping>10]
# filter out the bike stations of df_FB based on the multi index
flt_FB = df_FB.set_index(["datetime_return", "name_return"]).index.isin(grouping.index)
df_FB.loc[flt_FB, "is_mass_return"]=True
df_FB.is_mass_return.fillna(False, inplace=True)
df_FB.loc[df_FB.speed_kmh>50, "is_too_fast"]=True
df_FB.is_too_fast.fillna(False, inplace=True)
df_FB[['is_too_fast', 'is_mass_return']].value_counts()
df_FB.groupby("is_mass_return")['speed_kmh'].mean()
# correlation of 2 boolean columns
df_FB[['is_too_fast', 'is_mass_return']].corr()

df_FB['return_hour_int'] = df_FB.datetime_return.dt.hour

df_FB.is_mass_return.value_counts(normalize=True)


In [ ]:
#grouping = grouping[grouping>10]

In [ ]:
#grouping = grouping.reset_index()

In [ ]:
#grouping['weekday']= grouping.datetime_return.dt.weekday +1

In [ ]:
#grouping.weekday.value_counts()

In [ ]:
grouping

In [ ]:
#grouping = grouping.sort_values("datetime_return")

In [ ]:
#len(grouping)

In [ ]:
#grouping.to_csv("~/Downloads/grouping_DD.csv", index=False)

In [ ]:
df_FB[df_FB.is_mass_return==True].groupby("name_return").size().sort_values(ascending=False)

In [ ]:
df_FB[df_FB.is_mass_return==True].groupby("return_hour_int").size().sort_index()

In [ ]:
df_FB.is_mass_return.value_counts(normalize=True)

In [ ]:
len(df_DD[df_DD.is_too_fast == True])/len(df_DD)

In [ ]:
len(df_FB[df_FB.is_too_fast == True])/len(df_FB)

In [ ]:
len(df_DD[df_DD.is_too_fast==True])

In [ ]:
len(df_FB[df_FB.is_too_fast==True])

In [ ]:
df_DD = df_DD[df_DD.is_too_fast==False]
df_FB = df_FB[df_FB.is_too_fast==False]

In [ ]:
len(df_DD[df_DD.is_mass_return==True])

In [ ]:
len(df_FB[df_FB.is_mass_return==True])

In [ ]:

df_DD = df_DD[df_DD.is_mass_return==False]
df_FB = df_FB[df_FB.is_mass_return==False]


In [ ]:
len(df_DD)

In [ ]:
len(df_FB)

In [ ]:
df_DD = df_DD[original_cols]
df_FB = df_FB[original_cols]

In [ ]:
df_DD.name_return.value_counts(dropna=False)

In [ ]:
df_DD.name_rent.value_counts(dropna=False)

In [ ]:
# DD has latitude: 51° 02' 55''
# DD has longitude:  13° 44' 29''
# km_distance_from_station = 0
# lat - висота
# lng - ширина

# min_lat = df_DD.loc[ ~df_DD.station_number_return.isna(), "lat_return"].min() - km_distance_from_station/111
# max_lat = df_DD.loc[ ~df_DD.station_number_return.isna(), "lat_return"].max() + km_distance_from_station/111
# min_lng = df_DD.loc[ ~df_DD.station_number_return.isna(), "lng_return"].min() - km_distance_from_station/111
# max_lng = df_DD.loc[ ~df_DD.station_number_return.isna(), "lng_return"].max() + km_distance_from_station/111

# flt_DD_3 = (df_DD.lng_rent>min_lng) & (df_DD.lng_return>min_lng) & \
#         (df_DD.lng_rent<max_lng) & (df_DD.lng_return<max_lng) & \
#         (df_DD.lat_rent>min_lat) & (df_DD.lat_return>min_lat) & \
#         (df_DD.lat_rent<max_lat) & (df_DD.lat_return<max_lat)


# Save

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
df_DD.to_pickle(f'../data/nextbike/trips_Dresden {time}.pkl')
df_FB.to_pickle(f'../data/nextbike/trips_Freiburg {time}.pkl')

In [ ]:
time